In [1]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

<h1>Política de inventario optima para el problema multi-periodo y multi-producto con abastecimiento limitado por capacidad , lotes mínimos y métas de inventario</h1>
<p>El siguiente modelo propone una formulación matemática para resolver el problema de definir las órdenes de abastecimiento óptimas para un sistema de inventario donde existe un proveedor con capacidad limitada, lotes mínimos de compra/abastecimiento, y metas de inventario.</p>
<h2>Modelo</h2>

In [15]:
model = LpProblem(name="Inventory-policies-problem", sense=LpMinimize)

<h3>Conjuntos:</h3>
<p>$t$: periodos {1,2,..,n}</p>
<p>$p$: productos {1,2,..,m}</p>
<h3>Parámetros:</h3>
<p>$C$: Costo de colocar la orden, independiente de las cantidades de los productos $p$ comprados</p>
<p>$L$: Lead time del proveedor</p>
<p>$T_{t}$: Capacidad del proveedor durante el periodo $t$</p>
<p>$Q_{p}$: Inventario inicial del producto $p$</p>
<p>$H_{p}$: Costo de mantenimiento del inventario en la bodega del producto $p$</p>
<p>$S_{p}$: Lote mínimo de pedido para el producto $p$</p>
<p>$W_{p}$: Consumo de capacidad del proveedor del producto $p$</p>
<p>$F_{pt}$: Ventas pronosticadas del producto $p$ durante el periodo $t$</p>
<p>$A_{pt}$: Llegadas proyectadas del producto $p$ durante el periodo $t$</p>
<p>$O_{pt}$: Meta de inventario para el producto $p$ al final del periodo $t$</p>
<p>$D_{pt}$: Costo por no cumplor la meta de inventario del producto $p$ al final del periodo $t$</p>

In [16]:
product_count = 5
periods_count = 52
# declarando los conjuntos
t_sets = [x for x in range(periods_count)]
p_sets = [x for x in range(product_count)]
# declarando los parámetros
c_parameter = 2500
l_parameter = 12
t_parameter = np.array([500 for x in range(periods_count)])
q_parameter = np.random.normal(loc=25.0, scale=5, size=product_count)
h_parameter = np.array([0.1 for x in range(product_count)])
s_parameter = np.ones(product_count)*25
w_parameter = np.ones(product_count)*2
f_parameter = [np.random.normal(loc=5, scale=0.1, size=product_count) for x in t_sets]
a_parameter = [np.zeros(product_count) for x in t_sets]
o_parameter = [np.ones(product_count)*15 for x in t_sets]
d_parameter = [np.ones(periods_count)*x for x in p_sets]
m_parameter = np.sum(f_parameter)**2

<h3>Variables:</h3>
<p>$X_{pt}$ :Cantidad a ordenar del producto $p$ al comienzo del periodo $t$</p>
<p>$K_{pt}$ :Inventario on hand del producto $p$ al final del periodo $t$</p>
<p>$P_{t}$ :Cantidad de Pedidos a colocar al inicio del periodo $t$. Esta variable determina cuántos contenedores se van a despachar por el proveedor</p>
<p>$R_{pt}=\left\lbrace\begin{array}{c} 1 \text{ si el producto p será puesto en la orden de compra al inicio del periodo t} \\ 0 \text{ en otro caso} \end{array}\right.$</p>
<p>$E_{pt}=\left\lbrace\begin{array}{c} 1 \text{si se permitirá que el producto p quede por debajo de su meta de inventario al final del periodo t} \\ 0~\text{en otro caso} \end{array}\right.$</p>

In [17]:
x_variable = [[LpVariable(f'X(p={p};t={t})', lowBound=0.0, cat='Continuous') for p in p_sets] for t in t_sets]
k_variable = [[LpVariable(f'K(p={p};t={t})', cat='Continuous') for p in p_sets] for t in t_sets]
p_variable = [LpVariable(f'P(t={t})', lowBound=0, cat='Integer') for t in t_sets]
r_variable = [[LpVariable(f'R(p={p};t={t})', cat='Binary') for p in p_sets] for t in t_sets]
e_variable = [[LpVariable(f'E(p={p};t={t})', cat='Binary') for p in p_sets] for t in t_sets]

<h3>Función Objetivo</h3>

<p>La siguiente función objetivo suma el costo total de colocar una orden de abastecimiento, el costo total de mantener el inventario al final de la semana $t$ y, el costo total de penalidades por no cumplir con la meta de inventario</p>

<p>$$\max z = \sum_{p=1}^m\sum_{t=1}^n H_{pt}K_{pt} + \sum_{p=1}^m\sum_{t=1}^n CP_{pt} + \sum_{p=1}^m\sum_{t=1}^n D_{pt}E_{pr}$$</p>


In [18]:
model += lpSum([h_parameter[p] * k_variable[t][p] for t in t_sets for p in p_sets] +
               [c_parameter * p_variable[t] for t in t_sets] + 
               [d_parameter[p] * e_variable[t][p] for t in t_sets for p in p_sets])

<h3>Restricciones:</h3>

<p>El balance de inventario depende de 3 momentos en el horizonte de planeación:</p>
<ol>
    <li>En el periodo inicial, se tiene en cuenta el inventario inicial reportado;</li>
    <li>Durante el leadtime, se tiene en cuenta el inventario al final del periodo anterior como inventario inicial del periodo actual; y</li>
    <li>Despues del leadtime, donde se debe tener en cuenta las ordenes colocadas un leadtime antes</li>
</ol>
<p>Así, las siguientes restricciones modelan el comportamiento descrito:</p>

<p>$K_{pt} = Q_{p} + A_{pt} - F_{pt}$ for $t=1$</p>
<p>$K_{pt} = K_{pt-1} + A_{pt} - F_{pt}$ for $1 < t < L$</p>
<p>$K_{pt} = K_{pt-1} + A_{pt} - F_{pt} + X_{pt-L}$ for $t >= L$</p>

In [19]:
for p in p_sets:
    model += (k_variable[0][p] == q_parameter[p] + a_parameter[0][p] - f_parameter[0][p], 
              f'balanceInvProduct{p}_at_t={0}')

In [20]:
for p in p_sets:
    for t in t_sets[1:l_parameter]:
        model += (k_variable[t][p] == k_variable[t-1][p] + a_parameter[t][p] - f_parameter[t][p],
                  f'balanceInvProduct{p}_at_t={t}')

In [21]:
for p in p_sets:
    for t in t_sets[l_parameter:]:
        model += (k_variable[t][p] == k_variable[t-1][p] + a_parameter[t][p] - f_parameter[t][p] + x_variable[t-l_parameter][p],
                  f'balanceInvProduct{p}_at_t={t}')

<p>La siguiente restricción mantiene el inventario sobre la meta a partir del leadtime</p>
<p>$\sum_{p=L}^{n} X_{pt} >= O_{pt}$ for $t$</p>

In [22]:
for p in p_sets:
    for t in t_sets[l_parameter:]:
        model += (k_variable[t][p] >= o_parameter[t][p] ,f'Inventario del producto {p} sobre la meta en el periodo {t}')

<p>Ahora debemos obligar a que el modelo coloque pedidos sobre el lote mínimo por cada producto, lo que logramos con dos restricciones activadas de manera excluyente con una variable binaria y el parámetro M:</p>
<p>$X_{pt} <= MR_{pt}$</p>
<p>$X_{pt} >= S_{p}R_{pt}$</p>

In [23]:
for p in p_sets:
    for t in t_sets:
        model += (x_variable[t][p] <= m_parameter * r_variable[t][p] ,f'máxima cantidad a pedir del producto {p} en el periodo {t}')
        model += (x_variable[t][p] >= s_parameter[p] * r_variable[t][p] ,f'mínima cantidad a ordenar del producto {p} en el periodo {t} cumpliendo el lóte minimo')

<p>La cantidad a cargar un contenedor, no debe sobrepasar la capacidad del contenedor. Por ahora el parámetro $w_{p}$ nos permite contabilizar qué tanta carga podrá llevar cada contenedor, por lo que el pedido durante el periodo $t$ no debe sobrepasar la capacidad de un contenedor, multiplicado por la cantidad de contenedores</p>
<p>$\sum_{p}^n W_{p}X_{pt} <= T_{t}P_{t}$</p>

In [24]:
for t in t_sets:
    model += (lpSum([w_parameter[p] * x_variable[t][p] for p in p_sets]) <= t_parameter[t] * p_variable[t],
              f'Capacidad del proveedor/contenedores en el periodo{t}')

In [26]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/luispinilla/Documents/inventory_management_lp/lp_optimal_supply/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/zc/7j16sq9s55d3sj1zm53r4lb80000gn/T/4980b351f16947979716c44644fe10db-pulp.mps branch printingOptions all solution /var/folders/zc/7j16sq9s55d3sj1zm53r4lb80000gn/T/4980b351f16947979716c44644fe10db-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1037 COLUMNS
At line 4865 RHS
At line 5898 BOUNDS
At line 6679 ENDATA
Problem MODEL has 1032 rows, 1040 columns and 2267 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 12608.3 - 0.00 seconds
Cgl0003I 0 fixed, 52 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 757 rows, 762 columns (307 integer (255 of which binary)) and 1917 elements
Cbc0038I Initial state - 177 integers unsatisfied sum - 2.93431
Cbc0038I Pass

1

In [27]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 13223.382979573002
E(p=1;t=0): 0.0
E(p=1;t=1): 0.0
E(p=1;t=10): 0.0
E(p=1;t=11): 0.0
E(p=1;t=12): 0.0
E(p=1;t=13): 0.0
E(p=1;t=14): 0.0
E(p=1;t=15): 0.0
E(p=1;t=16): 0.0
E(p=1;t=17): 0.0
E(p=1;t=18): 0.0
E(p=1;t=19): 0.0
E(p=1;t=2): 0.0
E(p=1;t=20): 0.0
E(p=1;t=21): 0.0
E(p=1;t=22): 0.0
E(p=1;t=23): 0.0
E(p=1;t=24): 0.0
E(p=1;t=25): 0.0
E(p=1;t=26): 0.0
E(p=1;t=27): 0.0
E(p=1;t=28): 0.0
E(p=1;t=29): 0.0
E(p=1;t=3): 0.0
E(p=1;t=30): 0.0
E(p=1;t=31): 0.0
E(p=1;t=32): 0.0
E(p=1;t=33): 0.0
E(p=1;t=34): 0.0
E(p=1;t=35): 0.0
E(p=1;t=36): 0.0
E(p=1;t=37): 0.0
E(p=1;t=38): 0.0
E(p=1;t=39): 0.0
E(p=1;t=4): 0.0
E(p=1;t=40): 0.0
E(p=1;t=41): 0.0
E(p=1;t=42): 0.0
E(p=1;t=43): 0.0
E(p=1;t=44): 0.0
E(p=1;t=45): 0.0
E(p=1;t=46): 0.0
E(p=1;t=47): 0.0
E(p=1;t=48): 0.0
E(p=1;t=49): 0.0
E(p=1;t=5): 0.0
E(p=1;t=50): 0.0
E(p=1;t=51): 0.0
E(p=1;t=6): 0.0
E(p=1;t=7): 0.0
E(p=1;t=8): 0.0
E(p=1;t=9): 0.0
E(p=2;t=0): 0.0
E(p=2;t=1): 0.0
E(p=2;t=10): 0.0
E(p=2;t=11): 0.0
E(p=2;t=12)

In [25]:
model

Inventory-policies-problem:
MINIMIZE
52.0*E(p=1;t=0) + 52.0*E(p=1;t=1) + 52.0*E(p=1;t=10) + 52.0*E(p=1;t=11) + 52.0*E(p=1;t=12) + 52.0*E(p=1;t=13) + 52.0*E(p=1;t=14) + 52.0*E(p=1;t=15) + 52.0*E(p=1;t=16) + 52.0*E(p=1;t=17) + 52.0*E(p=1;t=18) + 52.0*E(p=1;t=19) + 52.0*E(p=1;t=2) + 52.0*E(p=1;t=20) + 52.0*E(p=1;t=21) + 52.0*E(p=1;t=22) + 52.0*E(p=1;t=23) + 52.0*E(p=1;t=24) + 52.0*E(p=1;t=25) + 52.0*E(p=1;t=26) + 52.0*E(p=1;t=27) + 52.0*E(p=1;t=28) + 52.0*E(p=1;t=29) + 52.0*E(p=1;t=3) + 52.0*E(p=1;t=30) + 52.0*E(p=1;t=31) + 52.0*E(p=1;t=32) + 52.0*E(p=1;t=33) + 52.0*E(p=1;t=34) + 52.0*E(p=1;t=35) + 52.0*E(p=1;t=36) + 52.0*E(p=1;t=37) + 52.0*E(p=1;t=38) + 52.0*E(p=1;t=39) + 52.0*E(p=1;t=4) + 52.0*E(p=1;t=40) + 52.0*E(p=1;t=41) + 52.0*E(p=1;t=42) + 52.0*E(p=1;t=43) + 52.0*E(p=1;t=44) + 52.0*E(p=1;t=45) + 52.0*E(p=1;t=46) + 52.0*E(p=1;t=47) + 52.0*E(p=1;t=48) + 52.0*E(p=1;t=49) + 52.0*E(p=1;t=5) + 52.0*E(p=1;t=50) + 52.0*E(p=1;t=51) + 52.0*E(p=1;t=6) + 52.0*E(p=1;t=7) + 52.0*E(p=1;t=8) + 52.